In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config

I0311 06:29:23.182612 140051521697600 file_utils.py:38] PyTorch version 1.4.0 available.
I0311 06:29:24.844568 140051521697600 modeling.py:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
Loading faiss with AVX2 support.
I0311 06:29:25.031044 140051521697600 modeling_bert.py:244] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [16]:
%load_ext autoreload
%autoreload 2
from awesome_glue.transforms import DAE, BackTrans
dae = DAE()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
DAE HUB is running under fairseq  0.6.1
| [src] dictionary: 49999 types
| [tgt] dictionary: 49999 types
| loading model(s) from /home/zhouyi/frequency/fsgec/out/models_pretrain/checkpoint9.pt


In [19]:
dae(["i am very happy", "hello b d e f", "c v g g h",'w yt f'])

<unk> c v g g h .
c v g g h
tensor([0, 0, 1, 2, 3, 4, 5, 5], dtype=torch.int32)
<unk> w f ...
hello b d e f
tensor([1, 0, 2, 1, 3], dtype=torch.int32)


['I am very happy .', 'Say hello b d e f .', 'c c v g g h .', 'b w f ...']

In [85]:
toy_data = load_data('TOY', 'spacy')

cache for TOY-spacy.data exists
*** load TOY-spacy.data from cache cost 4.42 seconds


In [249]:
from allennlp.data.batch import Batch
allennlp_collate([toy_data['data'][0][i] for i in range(2)])
toy_data['data'][0][0].as_tensor_dict()

{'sent': {'tokens': {'tokens': tensor([2866,   60,  337,    3,   93, 2195,  585])}},
 'label': tensor(0)}

In [187]:
embeds = torch.rand(5, 6, 300)
grads = torch.rand(5, 6, 300)
embedding_matrix = torch.rand(30000, 300)
src_tokens = torch.randint(vocab.get_vocab_size(), size=(5, 6))
src_tokens[0, 3:]=0
src_tokens[1, 1:]=0
src_tokens

tensor([[ 3801, 13419,  2493,     0,     0,     0],
        [14825,     0,     0,     0,     0,     0],
        [  862,   606, 16191, 13750,  9107,  5197],
        [ 1357, 14668,  2489, 16219,  3719,  9718],
        [ 3917, 11118, 14545,  5593,  4594,  2626]])

In [145]:
toy_data = load_data("TOY", "spacy")
vocab = toy_data['vocab']

cache for TOY-spacy.data exists
*** load TOY-spacy.data from cache cost 3.3 seconds


In [157]:
vocab.get_token_from_index(1)

'@@UNKNOWN@@'

In [159]:
from allennlpx.interpret.attackers.cached_searcher import CachedWordSearcher, CachedIndexSearcher
# searcher= CachedWordSearcher("nbrs.euc.top10.txt")
searcher= CachedIndexSearcher("nbrs.euc.top10.txt", word2idx=vocab.get_token_index, idx2word=vocab.get_token_from_index)

In [186]:
searcher.search("12131313")

([], [])

In [174]:
dir_dot_grad.new_zeros(dir_dot_grad.size(), dtype=torch.bool)

tensor([[[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, F

In [221]:
scores = dir_dot_grad
from luna import batch_pad

searcher = CachedIndexSearcher("euc-topk-10.txt",
                                word2idx=vocab.get_token_index,
                                idx2word=vocab.get_token_from_index)
src_tokens_lst = src_tokens.tolist()
idxes_to_mask = []
for bid in range(src_tokens.size(0)):
    for sid in range(src_tokens.size(1)):
        if src_tokens[bid][sid] == 0:
            idxes_to_mask.append([])
            continue
        _, idxs = searcher.search(src_tokens_lst[bid][sid])
        idxes_to_mask.append(idxs)
idxes_to_mask = src_tokens.new_tensor(batch_pad(idxes_to_mask, 0))
idxes_to_mask = idxes_to_mask.view(*src_tokens.size(), -1)
mask = scores.new_zeros(scores.size(), dtype=torch.bool)
mask.scatter_(dim=2, 
              index=idxes_to_mask, 
              src=mask_idxes.new_ones(mask_idxes.size(), dtype=torch.bool))
mask = ~mask
dir_dot_grad.masked_fill(mask, -19260817)

torch.Size([5, 6])


tensor([[[-1.5831e+00, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07],
         [-1.3415e+00, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07],
         [-1.9261e+07, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07],
         [-1.6058e+00, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07],
         [ 2.1660e+00, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07],
         [-5.3390e+00, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07]],

        [[-1.3436e+00, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07],
         [-4.5468e+00, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07],
         [ 3.5948e+00, -1.9261e+07, -1.9261e+07,  ..., -1.9261e+07,
          -1.9261e+07, -1.9261e+07],
         [-5.6091e+00, -1.9261e+07, -1.9261e+07,  ...

In [235]:
def bw_hook_layers(module, grad_in, grad_out):
    print('grad')
    print(grad_in)
    print(grad_out)

def fw_hook_layers(module, inputs, outputs):
    print(inputs)
    print(outputs)

# linear = torch.nn.Embedding(5, 3)
linear = torch.nn.Linear(5, 4)
linear.register_backward_hook(bw_hook_layers)
linear.register_forward_hook(fw_hook_layers)
    
linear(torch.randint(5, size=(7, 5)).float()).sum().backward()

(tensor([[2., 1., 4., 0., 4.],
        [4., 4., 3., 3., 4.],
        [2., 1., 3., 3., 1.],
        [4., 0., 4., 3., 4.],
        [3., 1., 4., 2., 3.],
        [2., 0., 1., 4., 2.],
        [1., 4., 4., 2., 2.]]),)
tensor([[ 0.0423,  1.6962, -0.6991,  1.0964],
        [ 0.9650,  2.1252, -0.9507,  2.1747],
        [-0.7851,  0.2252,  0.4127,  1.1427],
        [-0.8851,  0.7380,  0.1410,  1.7750],
        [-0.2944,  1.0871,  0.0547,  1.2855],
        [-1.5931, -0.4923, -0.2456,  1.8978],
        [ 0.3658,  1.9037, -0.7813,  1.6545]], grad_fn=<AddmmBackward>)
grad
(tensor([7., 7., 7., 7.]), None, tensor([[18., 18., 18., 18.],
        [11., 11., 11., 11.],
        [23., 23., 23., 23.],
        [17., 17., 17., 17.],
        [20., 20., 20., 20.]]))
(tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]]),)


In [203]:
def apply_constraint(src_tokens, scores):
    searcher = CachedIndexSearcher("nbrs.euc.top10.txt",
                                    word2idx=vocab.get_token_index,
                                    idx2word=vocab.get_token_from_index)
    mask = scores.new_zeros(scores.size(), dtype=torch.bool)
    src_token_lst = src_tokens.tolist()
    for bid in range(src_tokens.size(0)):
        for sid in range(src_tokens.size(1)):
            if src_tokens[bid][sid] == 0:
                break
            _, idxs = searcher.search(src_tokens_lst[bid][sid])
            for idx in idxs:
                mask[bid][sid][idx] = True
    mask = ~mask
    dir_dot_grad.masked_fill(mask, -19260817)
    
    
def hotflip(src_tokens, embeds, grads, embedding_matrix, k=3, constraint=None):
    k = min(k, src_tokens.size(1))
    
    # compute the direction vector dot the gradient 
    prev_embed_dot_grad = torch.einsum("bij,bij->bi", grads, embeds)
    new_embed_dot_grad = torch.einsum("bij,kj->bik", grads, embedding_matrix)
    dir_dot_grad = new_embed_dot_grad - prev_embed_dot_grad.unsqueeze(-1)
    dir_dot_grad[src_tokens == 0] = -19260817.
    
    # maybe some constraints
    if constraint == 'embed':
        apply_constraint(src_tokens, dir_dot_grad)
    
    # select the top-k step
    score_at_each_step, best_at_each_step = dir_dot_grad.max(2)
    _, best_positions = score_at_each_step.topk(k)
    
    # use the selected token index to replace the original one 
    adv_tokens = src_tokens.clone()
    src = best_at_each_step.gather(dim=1, index=best_positions)
    adv_tokens.scatter_(dim=1, index=best_positions, src=src)
    adv_tokens[src_tokens == 0] = 0
    return adv_tokens

In [202]:
hotflip(src_tokens,  embeds, grads, embedding_matrix)

tensor([[ 9338,  1292,  1264,     0,     0,     0],
        [23289,     0,     0,     0,     0,     0],
        [  862,  1264, 16191,  1264,  1264,  5197],
        [17234, 14668,  2489, 23289,  3719, 23300],
        [ 3917, 18620, 14545,  4146, 19632,  2626]])

In [37]:
def batch_index_select(inputs, indices):
    # inputs are like B*T*D
    # indices are like B*N
    # return B*N*D
    return torch.cat([torch.index_select(a, 0, i).unsqueeze(0) for a, i in zip(inputs, indices)])